In [2]:
# Phase Unwrapping Goldstein (Orientation Unwrapping)\n",
import queue
import imageio


#testimage\\n\",\n",
# img = imageio.imread('D:/Levchenko_Project_Readings/Pictures_of_fingerprints/Original_fingerprint.jpg')\\n\",\n",
#convert to grayscale\\n\"\n",
import cv2
img = cv2.imread('D:/Levchenko_Project_Readings/Pictures_of_fingerprints/Original_fingerprint.jpg', cv2.IMREAD_GRAYSCALE)

# img = imageio.imread('D:/Levchenko_Project_Readings/Pictures_of_fingerprints/Original_fingerprint_gray.jpg')
# print(\\\"gray image is read\\\")



# Calculates residue value from current, right, down, left
# How much is one cycle? what is half a cycle for a pixel? Is it 255?


# extract a smaller matrix for code to look better

def check_residue(pixel ,phase_image, cycle):

    if (pixel[0] >= len(phase_image)-1 or pixel[1] >= len(phase_image[0])-1):
        return None

    def sign(x):
        return int(1-(x<=0))

    #   check for residue
    residue = int(0)

    if( abs(-phase_image[pixel[0]][pixel[1]] + phase_image[pixel[0]][pixel[1]+1])> 0.5*cycle ):
        residue += sign(-phase_image[pixel[0]][pixel[1]] + phase_image[pixel[0]][pixel[1]+1])

    if(abs(phase_image[pixel[0]+1][pixel[1]+1] - phase_image[pixel[0]][pixel[1]+1])>0.5*cycle):
        residue += sign(phase_image[pixel[0]+1][pixel[1]+1] - phase_image[pixel[0]][pixel[1]+1])

    if(abs(phase_image[pixel[0]+1][pixel[1]] -phase_image[pixel[0]+1][pixel[1]+1])>0.5*cycle):
        residue += sign(phase_image[pixel[0]+1][pixel[1]] -phase_image[pixel[0]+1][pixel[1]+1])

    if (abs(phase_image[pixel[0]][pixel[1]]  - phase_image[pixel[0]+1][pixel[1]])>0.5*cycle):
        residue += sign(phase_image[pixel[0]][pixel[1]]  - phase_image[pixel[0]+1][pixel[1]])

    print("residue is ")
    print(residue%1)
    return residue%1




def Goldstein_unwrap(image ,ridge_freq):

    # Filtering with triangulation and ridge frequency (implement later)
    new_image = image

    # Here marked for branch element is [visited,charge]
    marked_for_branch = []
    zero_vec = []

    for j in range(0,len(new_image[0])):
        zero_vec.append([0,0])
    for i in range(0,len(new_image)):
        marked_for_branch.append(zero_vec)


    # Branch Cut Portion Using Priority Queue
    cuts_array = []
    initial_box_size = 3
    added_box_size = 0
    queue1 = queue.Queue(maxsize=0)
    queue1.put([0,0])
    residue = None

    print("gets to this checkpoint")

#interferogram scanned until residue found

#     residue = [0,0]



    #Branch cut algorithm
    def cutting(residue, queue1,added_box_size):
        print("calls cutting function")
        residuevalue = 0
        residuefound = False

        # Scans until residue is found by adding elements to a queue until residue is found
        while(residuefound == False):
            print("did not find residue")

            # check if there is an input residue if not start at the element popped in the list
            if (residue == None):
                print("current residue is null, so set to")
                #Pop the current element in the queue
                current = queue1.get()
                # change the residue to be the popped element for when exiting the loop
                residue = current
            #if not then empty the list and run the algorithm with current as assigned
            else:
                print("curent residue is")
                current = residue


            #Update the residue value and find out if it is found
            print(residue)
            print("which has value of")
            residuevalue = check_residue(current,new_image,255)
            print(residuevalue)
            residuefound = bool( residuevalue != 0 )

            # place the other 3 in the queue
            if(current[0] <= len(new_image)-2 and current[0] <= len(new_image[0])-2):
                print("gets qued")
                print([current[0]+1,current[1]])
                queue1.put([current[0]+1,current[1]])
                queue1.put([current[0]+1,current[1]+1])
                queue1.put([current[0],current[1]+1])

            #empty queue once found
            print("empty queue")
            queue1 = queue.Queue(maxsize=0)

            # Jumps out of loop once the residue is found
            print("found first residue")
            print(residuevalue)

            # Place a box of size 3 + added length around the residue and search for another residue
            for boxlen in range(1+int(added_box_size/2), initial_box_size+added_box_size):
                for boxwidth in range(1+int(added_box_size/2), initial_box_size+added_box_size):
                    #checks if the indexes are valid this is the problem?\n",
                    print("checking one unit within box")

                    try:
                        getresidue = check_residue([residue[0]-1-int(added_box_size/2)+boxlen,residue[1]-1-int(added_box_size/2)+boxwidth],new_image, 255)
                    except:
                        print("except")
                        continue

                    print("this is the cuts_array")
                    print(cuts_array)

                    if ([residue,getresidue,0] in cuts_array or [residue,getresidue,1] in cuts_array):
                        continue
                    # if residue is found
                    print("find other residue")
                    print("current residue has value")
                    print(getresidue)

                    if getresidue != 0:

                        # current problem, never jumps into this statement
                        print("passes criterion with none zero residue")

                        # If residue has opposite sign
                        if(residuevalue*getresidue< 0):
                            print("residue not same sign")
                            # locates the cut and designates as uncharged(0)
                            cuts_array.append([residue,getresidue, 0])
                            cutting([residue[0]-1-int(added_box_size/2)+boxlen,residue[1]-1-int(added_box_size/2)+boxwidth],queue1,added_box_size)


                        # If residue has same sign
                        if(residuevalue*getresidue > 0):
                            print("residue same sign")
                            # make the cut indicating it is charged
                            cuts_array.append([residue,getresidue, 1])

                            #Essentially a repeat of what happens above but there is a difference in calling and only cares about the negative sign
                            # box is moved to new residue and search continues until opposite residue is located
                            for boxlen in range(1+int(added_box_size/2), initial_box_size+added_box_size):
                                for boxwidth in range(1+int(added_box_size/2), initial_box_size+added_box_size):
                                    getresidue = check_residue([residue[0]-1-int(added_box_size/2)+boxlen,residue[1]-1-int(added_box_size/2)+boxwidth],new_image, 255)

                                    if ([residue,getresidue,0] in cuts_array or [residue,getresidue,1] in cuts_array):
                                        continue
                                    # if residue is found
#                                     intermed2 = !(boxlen ==1+int(int(added_box_size)/2) and boxwidth ==1+int(int(added_box_size)/2))
                                    if getresidue != 0:
                                        # If residue has opposite sign
                                        if(residuevalue*getresidue< 0):
                                            print("residue not same sign")
                                            # locates the cut and designates as uncharged(0)
                                            cuts_array.append([residue,getresidue, 0])
                                            cutting(None,queue1,added_box_size)


                            # If no more residues are left
                            #postincrement and run on the current pixel
                            print("increment box size within the same sign 2nd residue search for uncharged")
                            added_box_size+=2
                            if ( added_box_size+3 == min(len(image),len(image[0])) ):
                                return
                            cutting([residue[0]-1-int((added_box_size-2)/2)+boxlen,residue[1]-1-int((added_box_size-2)/2)+boxwidth],queue1,added_box_size)

                    print("next in the box")
                print("next line in box")

#             If  nothing is found for the second residue increase box size and call on current residue\n",
        print("increment box size")
        added_box_size+=2
        if ( added_box_size+3 == min(len(image),len(image[0])) ):
            return
        cutting()


    cutting(residue,queue1,added_box_size)
    print(cuts_array)
    return

#     print(\\\"function call is made\\\")            \n",

    # Integration of phase unwrapping avoiding the branch cuts\n",
#     unwrap_queue = Queue()\n",

#     unwrap_queue.put([0,0])\n",



Goldstein_unwrap(img ,4)



# 8-bit Grayscale convert\n",
# # function phi is the composite phase\n",

# def grayscale_8(x,y):\n",
#     return 255*(1+cos(phi(x,y)))/2\n",

#     # Reconstruct the Orientation field\n",

# # block_dim = 8\n",

# #     def S\n",

#     # m,n are coordinates for block, weighting function is applied to calculate the directions\n",
# def orient(m,n,weighting_function):\n",

#     #if there are singular minutia, use this as part of the function\n",
#     #Singular point influence\n",
#     # core 1, delta -1 index\n",
#     D_s = 0\n",
#     if (minutia_array[\"contains singular point index\"] == True):\n",
#         for z in (minutia_array):\n",
#             if (z[\"singular point index\"] == True):\n",
#                 D_s += (1/2)*z[\"core delta index\"]*arctan((n-minutia_array[\"n_si range of y in 8 by 8 block\"])/(m-minutia_array[\"m_si range of x in 8 by 8 block\"]))\n",



#     # This is for the non-singularity minutia\n",
#     u = 0\n",
#     for i in range(0, len(minutia_array)):\n",
#         u += cos(2*minutia_array[\"index for direction\"]) * weighting_function(minutia_array[\"relevant for weighting parts\"])\n",

#     v = 0\n",
#     for j in range(0, len(minutia_array)):\n",
#         v += cos(2*minutia_array[\"index for direction\"]) * weighting_function(minutia_array[\"relevant for weighting parts\"])\n",

#     return D_s + (1/2)*arctan(v/u)\n",

#     # Estimate Gradient of Continuous Phase\n",

# # Phase estimation\n",
# Phase = dict()\n",


# # Define Gradient phase at block (m,n)\n",
# def grad(m,n, ridge_freq):\n",
#     return 2*pi*ridge_freq*exp(1j*(orient(m,n,weighting_function)+pi/2))\n",

# def Spiral_Grad(m,n,ridge_freq):\\n\",\n",
#     #if there are singular minutia, use this as part of the function\n",
#     #Singular point influence\n",
#     Spiral = 0\n",
#     if (minutia_array[\"contains singular point index\"] == True):\n",
#         for z in (minutia_array):\n",
#             if (z[\"singular point index\"] == True):\n",
#                 Spiral += z[\"residue_sign\"]*arctan((n-minutia_array[\"n_si range of y in 8 by 8 block\"])/(m-minutia_array[\"m_si range of x in 8 by 8 block\"]))\n",


# def Cont_Grad(m,n,ridge_freq):\n",
#     grad(m,n,ridge_freq) - Spiral_Grad(m,n,ridge_freq)\n",


# def Cont_Phase(x,y):\n",
#     return Cont_Grad(m,n,ridge_freq).real*x + Cont_Grad(m,n,ridge_freq).imag*y + Phase[(m,n)]\n",


# def phase_estimate_border(image,x,y,m_prev,n_prev,m,n):\n",
#     return Cont_Grad(m_prev,n_prev,ridge_freq).real*x + Cont_Grad(m_prev,n_prev,ridge_freq).imag*y -(Cont_Grad(m,n,ridge_freq).real*x + Cont_Grad(m,n,ridge_freq).imag*y ) + Phase[(m_prev,n_prev)]\n",

# def phase_estimate(m,n):\n",
#     estimate = 0\n",
#     for i in range(0,8):\n",
#         estimate += phase_estimate_border(image,m(topleft_x)+i,n(topleft_y),m,n-1,m,n)\n",
#     for j in range(0,8):\n",
#         estimate += phase_estimate_border(image,m(topleft_x)+7,n(topleft_y)+j,m-1,n,m,n)\n",
#         estimate += phase_estimate_border(image,m(topleft_x)+7,n(topleft_y)+j,m+1,n,m,n)\n",
#     for k in range(0,8):\n",
#         estimate += phase_estimate_border(image,m(topleft_x)+k,n(topleft_y)+7,m,n+1,m,n)\n",


#     return estimate


gets to this checkpoint
calls cutting function
did not find residue
current residue is null, so set to
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has va

C:\Users\mastercljohnson\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in ubyte_scalars



[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
cu

this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]

0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array


...

0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
nex

0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array


residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has

residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
next line in box
did not find residue
curent residue is
[0, 0]
which has value of
residue is 
0
0
gets qued
[1, 0]
empty queue
found first residue
0
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has value
0
next in the box
checking one unit within box
residue is 
0
this is the cuts_array
[]
find other residue
current residue has

KeyboardInterrupt: 